In [13]:
!pip install chess
import chess
import chess.svg

In [23]:
from IPython.display import clear_output

with open("../data/raw/monster_chess_data.csv") as file:
    for i, line in enumerate(file):
        if(i == 0): continue
        fen = line.split(',')[0][1:-1]
        board = chess.Board(fen)
        display(chess.svg.board(board))
        input()
        clear_output()

